In [1]:
import numpy as np
import random
from IPython.display import clear_output
import gym


In [108]:
# Environment parameters
#env = gym.make('CartPole-v0')
#env = gym.make('Pong-v4')
#enviroment = gym.make("Taxi-v3").env
enviroment = gym.make('FrozenLake8x8-v1', is_slippery=False, map_name='8x8')
enviroment.render()
num_states = enviroment.observation_space.sample()
num_actions = enviroment.action_space.n
print(f"Number of states: {num_states}")
print(f"Observation space shape: {enviroment.observation_space.shape}")
print(f"Number of actions: {num_actions}")
#print(f"Action meanings: {enviroment.unwrapped.get_action_meanings()}")


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Number of states: 59
Observation space shape: ()
Number of actions: 4


In [109]:
alpha = 0.1
gamma = 0.6
epsilon = 0.1
q_table = np.zeros([enviroment.observation_space.n, enviroment.action_space.n])

In [117]:
alpha = 0.7
gamma = 0.5
epsilon = 0.5
q_table = np.zeros([enviroment.observation_space.n, enviroment.action_space.n])

In [133]:
num_of_episodes = 500#000

for episode in range(0, num_of_episodes):
    # Reset the enviroment
    state = enviroment.reset()

    # Initialize variables
    reward = 0
    terminated = False
    
    while not terminated:
        # Take learned path or explore new actions based on the epsilon
        if random.uniform(0, 1) < epsilon:
            action = enviroment.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        # Take action    
        next_state, reward, terminated, info = enviroment.step(action) 
        
        # Recalculate
        q_value = q_table[state, action]
        max_value = np.max(q_table[next_state])
        new_q_value = (1 - alpha) * q_value + alpha * (reward + gamma * max_value)
        
        # Update Q-table
        q_table[state, action] = new_q_value
        state = next_state
        
    if (episode + 1) % 100 == 0:
        clear_output(wait=True)
        print("Episode: {}".format(episode + 1))
        print(info)
        print(reward)
        print(action)
        enviroment.render()

print("**********************************")
print("Training is done!\n")
print("**********************************")

Episode: 500
{'prob': 1.0, 'TimeLimit.truncated': True}
0.0
2
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
**********************************
Training is done!

**********************************


In [134]:
### total_epochs, total_penalties = 0, 0
num_of_episodes = 5
patience = 100
total_penalties = 0
total_epochs = 0
for _ in range(num_of_episodes):
    state = enviroment.reset()
    epochs = 0
    penalties = 0
    reward = 0
    
    terminated = False
    
    while not terminated:
        action = np.argmax(q_table[state])
        state, reward, terminated, info = enviroment.step(action)
        print(reward)
        print(info)
        if reward <= 0:
            penalties += 1
        
        epochs += 1
        if (epochs + 1) % 1 == 0:
            clear_output(wait=True)
            print("Episode: {}".format(epochs+ 1))
            enviroment.render()

        if epochs > patience:
            terminated = True
    total_penalties += penalties
    total_epochs += epochs

print("**********************************")
print("Results")
print("**********************************")
print("Epochs per episode: {}".format(total_epochs / num_of_episodes))
print("Penalties per episode: {}".format(total_penalties / num_of_episodes))

Episode: 102
  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
**********************************
Results
**********************************
Epochs per episode: 101.0
Penalties per episode: 101.0


In [104]:
def print_policy(policy):
    interpretation = {
        0: "<",
        1: "v",
        2: ">",
        3: "^",
    }
    [print([interpretation[j] for j in i], "\n") for i in policy]
    return

def calculate_policy(q_table, dim):
    policy =np.zeros((dim,dim))
    for i in range(dim):
        for j in range(dim):
            #print(np.where(q_table[i*dim+j] == np.max(q_table[i*dim+j])))
            action =  np.where(q_table[i*dim+j] == np.max(q_table[i*dim+j]))[0]
            if type(action) == np.ndarray:
                action=action[0]
            policy[i][j] = action
    return policy

In [105]:
policy = calculate_policy(q_table, 4)

In [106]:
print(policy)

[[1. 2. 1. 0.]
 [1. 0. 1. 0.]
 [2. 1. 1. 0.]
 [0. 2. 2. 0.]]


In [107]:
print(print_policy(policy))

['v', '>', 'v', '<'] 

['v', '<', 'v', '<'] 

['>', 'v', 'v', '<'] 

['<', '>', '>', '<'] 

None
